In [2]:
import pandas as pd
from sklearn import datasets

from smote_variants.evaluation import evaluate_oversamplers, model_selection

In [3]:
tmp = pd.DataFrame({'a': [{'a': 1, 'b': 2}, {'a': 2, 'b': 3}]})

In [4]:
tmp['a'].apply(pd.Series)

,a,b
0,1,2
1,2,3


In [5]:
import numpy as np

np.isfinite(np.nan)

False

In [6]:
import inspect

In [7]:
list(inspect.signature(pd.DataFrame).parameters.keys())

['data', 'index', 'columns', 'dtype', 'copy']

In [8]:
import smote_variants
list(inspect.signature(smote_variants.SMOTE).parameters.keys())

['proportion',
 'n_neighbors',
 'nn_params',
 'ss_params',
 'n_jobs',
 'random_state',
 '_kwargs']

In [9]:
dataset_0 = datasets.load_breast_cancer()
dataset_0['name'] = 'breast_cancer'
dataset_1 = datasets.load_iris()
dataset_1['name'] = 'iris'
dataset_1['target'][dataset_1['target'] == 2] = 1

In [10]:
results = evaluate_oversamplers(datasets=[dataset_0, dataset_1],
                        oversamplers=[('smote_variants', 'SMOTE', {'n_jobs': 1, 'random_state': 5}),
                                        ('smote_variants', 'SMOTE', {'n_jobs': 1, 'n_neighbors': 9, 'random_state': 5})],
                        classifiers=[('sklearn.tree', 'DecisionTreeClassifier', {'random_state': 5}),
                                        ('sklearn.neighbors', 'KNeighborsClassifier', {'n_jobs': 1})],
                        cache_path='/home/gykovacs/smote_cache/',
                        #cache_path=None,
                        validator_params={'n_repeats': 2, 'n_splits': 5, 'random_state': 5},
                        scaler=('sklearn.preprocessing', 'StandardScaler', {}),
                        n_jobs=2,
                        parse_results=True)

2022-08-26 17:53:26.461779: processing dataset: breast_cancer


2022-08-26 17:53:26,591:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 9, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'SMOTE'}")
2022-08-26 17:53:26,595:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-08-26 17:53:26,596:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 9, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'SMOTE'}")
2022-08-26 17:53:26,597:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-08-26 17:53:26,599:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-08-26 17:53:26,601:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-08-26 17:53:26,839:INFO

2022-08-26 17:53:32.935343: processing dataset: iris


2022-08-26 17:53:32,978:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 9, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'SMOTE'}")
2022-08-26 17:53:32,982:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-08-26 17:53:32,984:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-08-26 17:53:32,985:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'SMOTE'}")
2022-08-26 17:53:32,987:INFO:SMOTE: simplex sampling with n_dim 2
2022-08-26 17:53:32,988:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-08-26 17:53:32,989:INFO:NearestNeighborsWithMet

In [16]:
all_scores = ['acc', 'sens', 'spec', 'ppv', 'npv', 'fpr', 'fdr',
                'fnr', 'bacc', 'gacc', 'f1', 'mcc', 'l', 'ltp', 'lfp', 'lfn',
                'ltn', 'lp', 'ln', 'uc', 'informedness', 'markedness', 'p_top20',
                'brier', 'log_loss', 'auc']

In [17]:
def _pivot_best_scores(pdf):
    """
    Pivot the best parameters for each oversampler and classifier

    Args:
        pdf (pd.DataFrame): the results averaged over folds

    Returns:
        pd.DataFrame: the pivoted results
    """
    results_dict_classifier = {}
    results_dict_oversampler = {}
    score_means = {}
    score_stds = {}
    for score in all_scores:
        tmp = pdf[pdf[score + '_mean'] == pdf[score + '_mean'].max()]

        results_dict_classifier[f'{score}_classifier_params'] = [None]
        results_dict_classifier[f'{score}_oversampler_params'] = [None]
        score_means[f'{score}_mean'] = [None]
        score_stds[f'{score}_std'] = [None]

        if len(tmp) > 0:
            results_dict_classifier[f'{score}_classifier_params'] =\
                                     [tmp.iloc[0]['classifier_params']]
            results_dict_classifier[f'{score}_oversampler_params'] =\
                                     [tmp.iloc[0]['oversampler_params']]
            score_means[f'{score}_mean'] = [tmp.iloc[0][f'{score}_mean']]
            score_stds[f'{score}_std'] = [tmp.iloc[0][f'{score}_std']]

    return pd.DataFrame({**results_dict_classifier,
                         **results_dict_oversampler,
                         **score_means,
                         **score_stds})


In [20]:
results.groupby(['dataset', 'oversampler', 'oversampler_module', 'classifier', 'classifier_module'])\
    .apply(_pivot_best_scores)\
    .reset_index(drop=False)

,dataset,oversampler,oversampler_module,classifier,classifier_module,level_5,acc_classifier_params,acc_oversampler_params,sens_classifier_params,sens_oversampler_params,...,ltn_std,lp_std,ln_std,uc_std,informedness_std,markedness_std,p_top20_std,brier_std,log_loss_std,auc_std
0,breast_cancer,SMOTE,smote_variants,DecisionTreeClassifier,sklearn.tree,0,{'random_state': 5},"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",{'random_state': 5},"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",...,10.452643,0.293488,0.153353,0.013415,0.052526,0.040359,0.072506,0.034362,1.186825,0.026263
1,breast_cancer,SMOTE,smote_variants,KNeighborsClassifier,sklearn.neighbors,0,{'n_jobs': 1},"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",{'n_jobs': 1},"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",...,7.624257,0.293488,0.153353,0.012255,0.056790,0.045269,0.072506,0.014180,0.465845,0.019780
2,iris,SMOTE,smote_variants,DecisionTreeClassifier,sklearn.tree,0,{'random_state': 5},"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",{'random_state': 5},"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",...,0.000000,0.000000,0.000000,None,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,iris,SMOTE,smote_variants,KNeighborsClassifier,sklearn.neighbors,0,{'n_jobs': 1},"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",{'n_jobs': 1},"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",...,0.000000,0.000000,0.000000,None,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
sampler, classifier = model_selection(dataset=dataset_0,
                        oversamplers=[('smote_variants', 'SMOTE', {'n_jobs': 1, 'random_state': 5}),
                                        ('smote_variants', 'SMOTE', {'n_jobs': 1, 'n_neighbors': 9, 'random_state': 5})],
                        classifiers=[('sklearn.tree', 'DecisionTreeClassifier', {'random_state': 5}),
                                        ('sklearn.neighbors', 'KNeighborsClassifier', {'n_jobs': 1})],
                        #cache_path='/home/gykovacs/smote_cache/',
                        cache_path=None,
                        validator_params={'n_repeats': 2, 'n_splits': 5, 'random_state': 5},
                        scaler=('sklearn.preprocessing', 'StandardScaler', {}),
                        n_jobs=2,
                        score='auc')

2022-08-26 17:53:33.723297: processing dataset: breast_cancer


2022-08-26 17:53:33,730:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'SMOTE'}")
2022-08-26 17:53:33,730:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'SMOTE'}")
2022-08-26 17:53:33,730:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-08-26 17:53:33,730:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-08-26 17:53:33,730:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-08-26 17:53:33,730:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-08-26 17:53:33,731:INFO

KeyError: "['level_4'] not found in axis"

In [ ]:
sampler

In [ ]:
[0, 1, 2] == [0, 1, 2]

True

In [ ]:
classifier

KNeighborsClassifier(n_jobs=1)

In [ ]:
results

,database,oversampler,classifier,oversampler_params_key,classifier_params_key,acc_mean,sens_mean,spec_mean,ppv_mean,npv_mean,...,ltn_std,lp_std,ln_std,uc_std,informedness_std,markedness_std,p_top20_std,brier_std,log_loss_std,auc_std
0,breast_cancer,SMOTE,DecisionTreeClassifier,"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",{'random_state': 5},0.934071,0.943995,0.917386,0.952162,0.908941,...,11.019668,0.293488,0.153353,0.013415,0.052526,0.040359,0.072506,0.019612,0.677388,0.026263
1,breast_cancer,SMOTE,DecisionTreeClassifier,"{'proportion': 1.0, 'n_neighbors': 9, 'nn_para...",{'random_state': 5},0.930539,0.942684,0.910299,0.947659,0.905863,...,10.452643,0.293488,0.153353,0.019710,0.077123,0.073411,0.072506,0.034362,1.186825,0.038561
2,breast_cancer,SMOTE,KNeighborsClassifier,"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",{'n_jobs': 1},0.925299,0.936972,0.905869,0.944265,0.895145,...,7.428414,0.293488,0.153353,0.012255,0.056790,0.045269,0.072506,0.014180,0.465845,0.017591
3,breast_cancer,SMOTE,KNeighborsClassifier,"{'proportion': 1.0, 'n_neighbors': 9, 'nn_para...",{'n_jobs': 1},0.923537,0.935563,0.903488,0.942887,0.893544,...,7.624257,0.293488,0.153353,0.010445,0.052173,0.043289,0.072506,0.015354,0.514612,0.019780
4,iris,SMOTE,DecisionTreeClassifier,"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",{'random_state': 5},1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,iris,SMOTE,DecisionTreeClassifier,"{'proportion': 1.0, 'n_neighbors': 9, 'nn_para...",{'random_state': 5},1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,iris,SMOTE,KNeighborsClassifier,"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",{'n_jobs': 1},1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,iris,SMOTE,KNeighborsClassifier,"{'proportion': 1.0, 'n_neighbors': 9, 'nn_para...",{'n_jobs': 1},1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
all_scores = ['acc', 'sens', 'spec', 'ppv', 'npv', 'fpr', 'fdr',
                'fnr', 'bacc', 'gacc', 'f1', 'mcc', 'l', 'ltp', 'lfp', 'lfn',
                'ltn', 'lp', 'ln', 'uc', 'informedness', 'markedness', 'p_top20',
                'brier', 'log_loss', 'auc']

def aggregate_folds(pdf):
    for score in all_scores:
        pdf[score] = pdf['scores'].apply(lambda x: x[score])
    pdf['database'] = pdf['fold_descriptor'].apply(lambda x: x['name'])
    pdf['fold_idx'] = pdf['fold_descriptor'].apply(lambda x: x['fold_idx'])
    pdf['repeat_idx'] = pdf['fold_descriptor'].apply(lambda x: x['repeat_idx'])
    pdf['oversampler_params'] = pdf['oversampler']
    pdf['oversampler'] = pdf['oversampler_params'].apply(lambda x: x['class_name'])
    pdf['oversampler_params_key'] = pdf['oversampler_params'].apply(str)
    pdf['classifier_params_key'] = pdf['classifier_params'].apply(str)
    return pdf

In [ ]:
pdf_prep = aggregate_folds(pdf)

In [ ]:
import numpy as np
pdf_avg = pdf_prep.groupby(['database', 'oversampler', 
                            'classifier', 'oversampler_params_key', 
                            'classifier_params_key']).agg(**{score + '_mean': ('score', 'mean') for score in all_scores})

In [ ]:
import numpy as np
pdf_avg = pdf_prep.groupby(['database', 'oversampler', 
                            'classifier', 'oversampler_params_key', 
                            'classifier_params_key']).agg(**{**{score + '_mean': (score, 'mean') for score in all_scores},
                                                           **{score + '_std': (score, 'std') for score in all_scores}})

In [ ]:
pdf_avg

acc_mean  \
database      oversampler classifier             oversampler_params_key                             classifier_params_key             
breast_cancer SMOTE       DecisionTreeClassifier {'proportion': 1.0, 'n_neighbors': 5, 'nn_param... {'random_state': 5}    0.934071   
                                                 {'proportion': 1.0, 'n_neighbors': 9, 'nn_param... {'random_state': 5}    0.930539   
                          KNeighborsClassifier   {'proportion': 1.0, 'n_neighbors': 5, 'nn_param... {'n_jobs': 1}          0.925299   
                                                 {'proportion': 1.0, 'n_neighbors': 9, 'nn_param... {'n_jobs': 1}          0.923537   
iris          SMOTE       DecisionTreeClassifier {'proportion': 1.0, 'n_neighbors': 5, 'nn_param... {'random_state': 5}    1.000000   
                                                 {'proportion': 1.0, 'n_neighbors': 9, 'nn_param... {'random_state': 5}    1.000000   
                          KNeighborsClassifier   {'proportion': 1.0, 'n_neighbors': 5, 'nn_param... {'n_jobs': 1}          1.000000   
                                                 {'proportion': 1.0, 'n_neighbors': 9, 'nn_param... {'n_jobs': 1}          1.000000   

                                                                                                                           sens_mean  \
database      oversampler classifier             oversampler_params_key                             classifier_params_key              
breast_cancer SMOTE       DecisionTreeClassifier {'proportion': 1.0, 'n_neighbors': 5, 'nn_param... {'random_state': 5}     0.943995   
                                                 {'proportion': 1.0, 'n_neighbors': 9, 'nn_param... {'random_state': 5}     0.942684   
                          KNeighborsClassifier   {'proportion': 1.0, 'n_neighbors': 5, 'nn_param... {'n_jobs': 1}           0.936972   
                                                 {'proportion': 1.0, 'n_neighbors': 9, 'nn_param... {'n_jobs': 1}           0.935563   
iris          SMOTE       DecisionTreeClassifier {'proportion': 1.0, 'n_neighbors': 5, 'nn_param... {'random_state': 5}     1.000000   
                                                 {'proportion': 1.0, 'n_neighbors': 9, 'nn_param... {'random_state': 5}     1.000000   
                          KNeighborsClassifier   {'proportion': 1.0, 'n_neighbors': 5, 'nn_param... {'n_jobs': 1}           1.000000   
                                                 {'proportion': 1.0, 'n_neighbors': 9, 'nn_param... {'n_jobs': 1}           1.000000   

                                                                                                                           spec_mean  \
database      oversampler classifier             oversampler_params_key                             classifier_params_key              
breast_cancer SMOTE       DecisionTreeClassifier {'proportion': 1.0, 'n_neighbors': 5, 'nn_param... {'random_state': 5}     0.917386   
                                                 {'proportion': 1.0, 'n_neighbors': 9, 'nn_param... {'random_state': 5}     0.910299   
                          KNeighborsClassifier   {'proportion': 1.0, 'n_neighbors': 5, 'nn_param... {'n_jobs': 1}           0.905869   
                                                 {'proportion': 1.0, 'n_neighbors': 9, 'nn_param... {'n_jobs': 1}           0.903488   
iris          SMOTE       DecisionTreeClassifier {'proportion': 1.0, 'n_neighbors': 5, 'nn_param... {'random_state': 5}     1.000000   
                                                 {'proportion': 1.0, 'n_neighbors': 9, 'nn_param... {'random_state': 5}     1.000000   
                          KNeighborsClassifier   {'proportion': 1.0, 'n_neighbors': 5, 'nn_param... {'n_jobs': 1}           1.000000   
                                                 {'proportion': 1.0, 'n_neighbors': 9, 'nn_param... {'n_jobs': 1}           1.000000   

                                                  

In [ ]:
def pivot_best_scores(pdf):
    base_params = ['database', 'oversampler', 'classifier']
    results_dict_classifier = {}
    results_dict_oversampler = {}
    score_means = {}
    score_stds = {}
    for score in all_scores:
        tmp = pdf[pdf[score + '_mean'] == pdf[score + '_mean'].max()]
        if len(tmp) > 0:
            results_dict_classifier[f'{score}_classifier_params'] = [tmp.iloc[0]['classifier_params_key']]
            results_dict_classifier[f'{score}_oversampler_params'] = [tmp.iloc[0]['oversampler_params_key']]
            score_means[f'{score}_mean'] = [tmp.iloc[0][f'{score}_mean']]
            score_stds[f'{score}_std'] = [tmp.iloc[0][f'{score}_std']]
        else:
            results_dict_classifier[f'{score}_classifier_params'] = [None]
            results_dict_classifier[f'{score}_oversampler_params'] = [None]
            score_means[f'{score}_mean'] = [None]
            score_stds[f'{score}_std'] = [None]

    return pd.DataFrame({**results_dict_classifier,
                         **results_dict_oversampler,
                         **score_means,
                         **score_stds})

In [ ]:
results\
    .reset_index(drop=False)\
    .groupby(['database', 'oversampler', 'classifier'])\
    .apply(pivot_best_scores)\
    .reset_index(drop=False)\
    [['database', 'oversampler', 'classifier', 'auc_mean', 'auc_std']]

,database,oversampler,classifier,auc_mean,auc_std
0,breast_cancer,SMOTE,DecisionTreeClassifier,0.930691,0.026263
1,breast_cancer,SMOTE,KNeighborsClassifier,0.960231,0.019780
2,iris,SMOTE,DecisionTreeClassifier,1.000000,0.000000
3,iris,SMOTE,KNeighborsClassifier,1.000000,0.000000


In [ ]:
from sklearn.neighbors._classification import KNeighborsClassifier

In [ ]:
KNeighborsClassifier.__module__

'sklearn.neighbors._classification'

In [ ]:
pdf_avg.reset_index(drop=False)

,database,oversampler,classifier,oversampler_params_key,classifier_params_key,acc,sens,spec,ppv,npv,...,ltn,lp,ln,uc,informedness,markedness,p_top20,brier,log_loss,auc
0,breast_cancer,SMOTE,DecisionTreeClassifier,"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",{'random_state': 5},0.934071,0.943995,0.917386,0.952162,0.908941,...,-149.545905,-33.281311,-41.858889,0.102212,0.861381,0.861103,0.368182,0.065929,2.277139e+00,0.930691
1,breast_cancer,SMOTE,DecisionTreeClassifier,"{'proportion': 1.0, 'n_neighbors': 9, 'nn_para...",{'random_state': 5},0.930539,0.942684,0.910299,0.947659,0.905863,...,-148.474669,-33.281311,-41.858889,0.100870,0.852983,0.853522,0.368182,0.069461,2.399134e+00,0.926491
2,breast_cancer,SMOTE,KNeighborsClassifier,"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",{'n_jobs': 1},0.925299,0.936972,0.905869,0.944265,0.895145,...,-148.126537,-33.281311,-41.858889,0.095950,0.842841,0.839410,0.368182,0.059977,9.545901e-01,0.958374
3,breast_cancer,SMOTE,KNeighborsClassifier,"{'proportion': 1.0, 'n_neighbors': 9, 'nn_para...",{'n_jobs': 1},0.923537,0.935563,0.903488,0.942887,0.893544,...,-147.835020,-33.281311,-41.858889,0.094792,0.839052,0.836431,0.368182,0.059834,8.974790e-01,0.960231
4,iris,SMOTE,DecisionTreeClassifier,"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",{'random_state': 5},1.000000,1.000000,1.000000,1.000000,1.000000,...,-23.025851,-8.109302,-10.986123,NaN,1.000000,1.000000,0.000000,0.000000,9.992007e-16,1.000000
5,iris,SMOTE,DecisionTreeClassifier,"{'proportion': 1.0, 'n_neighbors': 9, 'nn_para...",{'random_state': 5},1.000000,1.000000,1.000000,1.000000,1.000000,...,-23.025851,-8.109302,-10.986123,NaN,1.000000,1.000000,0.000000,0.000000,9.992007e-16,1.000000
6,iris,SMOTE,KNeighborsClassifier,"{'proportion': 1.0, 'n_neighbors': 5, 'nn_para...",{'n_jobs': 1},1.000000,1.000000,1.000000,1.000000,1.000000,...,-23.025851,-8.109302,-10.986123,NaN,1.000000,1.000000,0.000000,0.000000,9.992007e-16,1.000000
7,iris,SMOTE,KNeighborsClassifier,"{'proportion': 1.0, 'n_neighbors': 9, 'nn_para...",{'n_jobs': 1},1.000000,1.000000,1.000000,1.000000,1.000000,...,-23.025851,-8.109302,-10.986123,NaN,1.000000,1.000000,0.000000,0.000000,9.992007e-16,1.000000


In [ ]:
pdf['score'] = pdf['scores'].apply(lambda x: x['auc'])

In [ ]:
pdf[pdf['score'] == pdf['score'].max()].iloc[0]

fold_descriptor           {'db_n': 150, 'db_n_attr': 4, 'imbalance_ratio...
oversampler                                                           SMOTE
runtime                                                            0.001434
y_pred                    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
y_test                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...
scores                    {'tp': 20, 'tn': 10, 'fp': 0, 'fn': 0, 'p': 20...
classifier                                           DecisionTreeClassifier
classifier_params                                       {'random_state': 5}
acc                                                                     1.0
sens                                                                    1.0
spec                                                                    1.0
ppv                                                                     1.0
npv                                                                     1.0
fpr         

In [ ]:
from smote_variants import MulticlassOversampling

In [ ]:
mo = MulticlassOversampling('SUNDO')

In [ ]:
from smote_variants import SUNDO

In [ ]:
import inspect

In [ ]:
'proportion' in list(inspect.signature(SUNDO).parameters.keys())

False